In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload


import sys, os, inspect
import sys
sys.path.append('/Users/emigardiner/VICO/pjams-ionization/pjams/')

from zeusmp_snapshot_reader import read_zeusmp_snapshot
from zeusmp_snapshot_reader import ScaleFactors
from snapshot import snapshot 
from basic_snapshot import basic_snapshot
import plot
import flux

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
from matplotlib.colors import TwoSlopeNorm
from matplotlib.lines import Line2D

from PIL import Image, ImageDraw, ImageFont
from IPython.display import Image as DisplayImage

In [ ]:
VICO_loc = '/Users/emigardiner/VICO/pjams-ionization'

# INPUTS
freqs = np.array([.01, .05, .1, .5, 1, 5.3, 23, 43, 100, 230 ]) # GHz
freqs *= 10**9 # Hz

r_kpc = 1
heights_and_scales = np.load(VICO_loc+'/Data/heights_and_scales.npz')
scales = heights_and_scales['scales'] # AU
heights = heights_and_scales['heights'] # AU  

colors = plot.COLORS

# Read in Snapshots

In [ ]:
nums = np.array([4, 9, 21, 39, 54, 68, 94])
years = np.array(['4,000 yrs', '9,000 yrs', '21,000 yrs', '39,000 yrs', '54,000 yrs', '68,000 yrs', '94,000 yrs'])
masss = np.array([r'1.4 M$_\odot$', r'2 M$_\odot$', r'4 M$_\odot$',
                   r'8 M$_\odot$', r'12 M$_\odot$', r'16 M$_\odot$',
                   r'24 M$_\odot$'])

In [ ]:
snaps = np.empty_like(nums, dtype=snapshot)
for ii, num in enumerate(nums):
    snaps[ii] = snapshot(snap=num, name = ('Snap%03d_n' % num), read_zeusmp = False)
    # shot[ii].load_shock_variables()
    snaps[ii].load_fluxes()

draw_flux_vs_height()

In [ ]:
def draw_flux_vs_height(ax, xx, yy, labels, fmt, alpha=0.6, colors=colors, ms=None):
    # colors = cm.gist_rainbow_r(np.linspace(0, 1, len(snaps)))
    for ii in range(len(yy)):
        ax.plot(xx, yy[ii], fmt, color=colors[ii], label = (labels[ii]), alpha=alpha, ms=ms)



# All-Res

In [ ]:
rnums = np.array([9, 21, 39, 54])
ryears = years[1:len(rnums)+1]
rmasss = masss[1:len(rnums)+1]
snaps_lr, snaps_mr, snaps_hr = flux.prep_res_snapshots()

mass_lr = np.array(['2 M$_\odot$ lr', '4 M$_\odot$ lr', '8 M$_\odot$ lr', '12 M$_\odot$ lr',])
mass_mr = np.array(['2 M$_\odot$ mr', '4 M$_\odot$ mr', '8 M$_\odot$ mr', '12 M$_\odot$ mr',])
mass_hr = np.array(['2 M$_\odot$ hr', '4 M$_\odot$ hr', '8 M$_\odot$ hr', '12 M$_\odot$ hr',])
# print(mass_lr, mass_mr, mass_hr)

## Res Z Segments

In [ ]:
debug = False
targets=np.linspace(0,26000,27)
if debug: print(targets)

zseg_lr, zval_lr, zmid_lr = flux.get_z_segments(snaps_lr[0].x1, targets)
if debug: print(f"{zseg_lr=} \n{zval_lr=} \n{zmid_lr=}")

zseg_mr, zval_mr, zmid_mr = flux.get_z_segments(snaps_mr[0].x1, targets)
if debug: print(f"{zseg_mr=} \n{zval_mr=} \n{zmid_mr=}")

zseg_hr, zval_hr, zmid_hr = flux.get_z_segments(snaps_hr[0].x1, targets)
if debug: print(f"{zseg_hr=} \n{zval_hr=} \n{zmid_hr=}")

## Calculate Flux Profiles

### Without scaling

In [ ]:
# NORM = False
# norm_label = 'scaled to 1000 au' if NORM else 'unscaled pixel regions'

# for ratio in [True, False]:
#     for ss, snap in enumerate(snaps_lr):
#         print(snap.name, 'ratio=',ratio)
#         flux.make_flux_profiles(snap, zseg_lr, ratio=ratio, debug=False, norm=NORM)

#     for ss, snap in enumerate(snaps_mr):
#         print(snap.name, 'ratio=',ratio)
#         flux.make_flux_profiles(snap, zseg_mr, ratio=ratio, debug=False, norm=NORM)

#     for ss, snap in enumerate(snaps_hr):
#         print(snap.name, 'ratio=',ratio)
#         flux.make_flux_profiles(snap, zseg_hr, ratio=ratio, debug=False, norm=NORM)

## Plot Flux vs Height AllRes

In [ ]:
def plot_res_flux_vs_height( 
        snaps_lr, snaps_mr, snaps_hr, 
        xx_lr, xx_mr, xx_hr, 
        mass_labels,  
        ff_arr = [5,9],
        ymin0 = None, ymax0 = None, ymin1 = None, ymax1 = None, 
        xscale='log', ax0_text = None, ax1_text=None, xx_text = 0.02, yy_text = 0.02, 
        leg_anchor = (1, 0), colors=colors, ms=5,
        add_custom_leg = False,
):
    fig, axs = plot.figax_double(sharex=True,
                         xscale=xscale, nrows=2, ncols=2, height=7.5)

    all_snaps = np.array([snaps_lr, snaps_mr, snaps_hr])
    all_xx = np.array([xx_lr, xx_mr, xx_hr])
    all_fmt = np.array([':x', '--+', '-o',])
    none_labels = np.repeat('', len(snaps_lr))

    # first row: no ratio
    for ii, ax in enumerate(axs[0,:]):
        ff = ff_arr[ii]

        for jj, snaps in enumerate(all_snaps):
            xx = all_xx[jj]
            fmt = all_fmt[jj]
            if ii==0 and jj==0: labels = mass_labels
            else: labels = none_labels

            yy = np.zeros((len(snaps), len(xx)))
            for ss, snap in enumerate(snaps):
                temp = snap.flux_prof
                yy[ss] = temp[ff,:-1]
            draw_flux_vs_height(ax, xx, yy, labels=labels, fmt=fmt, colors=colors, ms=ms)

        # ylabel = r'$S_\nu$ (%s GHz) [mJy]' % str(freqs[ff]/10**9)
        ylabel = '$S_{\\nu=\mathrm{%s\ GHz}}\ \mathrm{per\ 1000\ au\ (mJy)}$' % str(freqs[ff]/10**9)

        ax.set_ylabel(ylabel)
        if ax0_text is not None:
            ax.text(xx_text, yy_text, ax0_text, transform=ax.transAxes, 
                    weight='bold', horizontalalignment='right')
            
    if ymin0 is not None and ymax0 is not None:
        axs[0,0].set_ylim(ymin0, ymax0)

    axs[0,1].sharey(axs[0,0])

    # second row: ratio
    for ii, ax in enumerate(axs[1,:]):
        ff = ff_arr[ii]

        for jj, snaps in enumerate(all_snaps):
            xx = all_xx[jj]
            fmt = all_fmt[jj]
            if ii==1 and jj==2: labels = mass_labels
            else: labels = none_labels

            yy = np.zeros((len(snaps), len(xx)))
            for ss, snap in enumerate(snaps):
                temp = snap.flux_prof_ratio
                yy[ss] = temp[ff,:-1]
            draw_flux_vs_height(ax, xx, yy, labels=labels, fmt=fmt, colors=colors, ms=ms)

        # ylabel = r'$S_\nu$ (%s GHz) [mJy]' % str(freqs[ff]/10**9)
        ylabel = '$S_{\\nu=\mathrm{%s\ GHz}}\ \mathrm{per\ 1000\ au\ (mJy)}$' % str(freqs[ff]/10**9)

        ax.set_ylabel(ylabel)
        ax.set_xlabel('$z$ (au)')
        if ax1_text is not None:
            ax.text(xx_text, yy_text, ax1_text, transform=ax.transAxes, 
                    weight='bold', horizontalalignment='right')

    if ymin1 is not None and ymax1 is not None:
        axs[1,0].set_ylim(ymin1, ymax1)

    axs[1,1].sharey(axs[1,0])
    axs[1,1].legend(bbox_to_anchor = leg_anchor, bbox_transform=ax.transAxes, loc='lower right')


    fig.subplots_adjust(hspace=0)
    return fig
    
res_legend_elements = [
            Line2D([0], [0], color='k', marker='x', linestyle=':', label='low-res'),
            Line2D([0], [0], color='k', marker='+', linestyle='--',  label='mid-res'),
            Line2D([0], [0], color='k', marker='o', linestyle='-',  label='high-res')]

### With Scaling
load/make flux profiles

In [ ]:
NORM = True
save=True
if NORM is False: save = False

norm_label = 'scaled to 1000 au' if NORM else 'unscaled pixel regions'

for ratio in [True, False]:
    for ss, snap in enumerate(snaps_lr):
        # print(snap.name, 'ratio=',ratio)
        flux.load_fluxprof(snap, ratio)
        # flux.make_flux_profiles(snap, zseg_lr, ratio=ratio, debug=False, norm=NORM, save=True)

    for ss, snap in enumerate(snaps_mr):
        # print(snap.name, 'ratio=',ratio)
        flux.load_fluxprof(snap, ratio)
        # flux.make_flux_profiles(snap, zseg_mr, ratio=ratio, debug=False, norm=NORM, save=True)

    for ss, snap in enumerate(snaps_hr):
        # print(snap.name, 'ratio=',ratio)
        flux.load_fluxprof(snap, ratio)
        # flux.make_flux_profiles(snap, zseg_hr, ratio=ratio, debug=False, norm=NORM, save=save)

save linear figure!

In [ ]:
print(snaps_hr[-1].__dict__.keys())

In [ ]:

fig = plot_res_flux_vs_height( 
        snaps_lr, snaps_mr, snaps_hr, 
        xx_lr=zmid_lr[:-1], xx_mr=zmid_mr[:-1], xx_hr=zmid_hr[:-1], 
        mass_labels=masss[1:len(rnums)+1],
        xscale='linear', ax0_text='no cooling', ax1_text='with cooling',
        ymin0 = 10**-7, ymax0 = 10**2.5, ymin1 = 10**-7, ymax1 = 10**-1, 
        xx_text=0.99, yy_text=0.025, 
        leg_anchor=(1, 0.07), colors=colors, ms=0) #, add_custom_leg=True)

fig.axes[2].legend(handles=res_legend_elements, bbox_to_anchor = (1,0.07), loc='lower right')
        # bbox_transform=ax.transAxes, 
# fig.tight_layout()
fig.savefig('/Users/emigardiner/VICO/pjams-ionization/figures/flux_profiles/allres_fluxprofile.png', 
        bbox_inches='tight', dpi=200)

# fig.text(0,0, norm_label, fontsize=14)
# fig.savefig('/Users/emigardiner/VICO/pjams-ionization/figures/flux_profiles/allres_fluxprofile_norm_lin.png', dpi=200)

save log figure!

In [ ]:
fig = plot_res_flux_vs_height( 
        snaps_lr, snaps_mr, snaps_hr, 
        xx_lr=zmid_lr[:-1], xx_mr=zmid_mr[:-1], xx_hr=zmid_hr[:-1], 
        mass_labels=masss[1:len(rnums)+1],
        xscale='log', ax0_text='no cooling', ax1_text='with cooling',
        ymin0 = 10**-7, ymax0 = 10**2.5, ymin1 = 10**-7, ymax1 = 10**-1, 
        xx_text=0.99, yy_text=0.025, 
        leg_anchor=(1.0, 0.07), colors=colors)
fig.savefig('/Users/emigardiner/VICO/pjams-ionization/figures/flux_profiles/allres_fluxprofile_log.png', dpi=200)
fig.text(0,0, norm_label, fontsize=14)
fig.tight_layout()

# 3 Mid-Res Only, New Code

calculate deltas

In [ ]:
for ii, snap in enumerate(snaps):
    snap.calculate_deltas()

load/make flux profiles

In [ ]:
NORM = True
norm_label = 'scaled to 1000 au' if NORM else 'unscaled pixel regions'


for ratio in [True, False]:
    for ss, snap in enumerate(snaps):
        print(snap.name, 'ratio=',ratio)
        flux.load_fluxprof(snap, ratio)
        # flux.make_flux_profiles(snap, zseg_mr, ratio=ratio, debug=False, norm=NORM, save=True)

# Plot mid res flux vs height separate cases

In [ ]:
def plot_mid_flux_vs_height_case( 
        snaps_mr,  
        xx_mr, 
        mass_labels,  
        ff_arr = [5,9], case='A',
        ymin = None, ymax = None, 
        xscale='log', ax0_text = None, ax1_text=None, xx_text = 0.02, yy_text = 0.02, 
        leg_anchor = (1, 0),
        colors=colors, legax=[1,1], all_fmt = ['-', '-'],
):
    fig, axs = plot.figax_single(sharex=True,
                         xscale=xscale, nrows=2, height=7)

    all_snaps = np.array([snaps_mr, ])
    all_xx = np.array([ xx_mr, ])
    none_labels = np.repeat('', len(snaps_mr))

    axs[1].set_xlabel('$z$ (au)')

    # first row: no ratio
    for ii, ax in enumerate(axs[:]):
        ff = ff_arr[ii]

        for jj, snaps in enumerate(all_snaps):
            xx = all_xx[jj]
            fmt = all_fmt[0]
            labels = mass_labels

            yy = np.zeros((len(snaps), len(xx)))
            for ss, snap in enumerate(snaps):
                temp = snap.flux_prof if case=='A' else snap.flux_prof_ratio
                yy[ss] = temp[ff,:-1]
            draw_flux_vs_height(ax, xx, yy, labels=labels, fmt=fmt, colors=colors, alpha=0.9)

        ylabel = '$S_{\\nu=\mathrm{%s\ GHz}}\ \mathrm{per\ 1000\ au\ (mJy)}$' % str(freqs[ff]/10**9)
        # ylabel = '$S_{\\nu=\mathrm{%s\ GHz}}$ per 1000 au (mJy)' % str(freqs[ff]/10**9)

        ax.set_ylabel(ylabel)
        if ax0_text is not None:
            ax.text(xx_text, yy_text, ax0_text, transform=ax.transAxes, 
                    weight='bold', horizontalalignment='right')
        if ymin is not None and ymax is not None:
            ax.set_ylim(ymin, ymax)
    # axs[0,1].sharey(axs[0,0])
    axs[1].legend(bbox_to_anchor = leg_anchor, bbox_transform=ax.transAxes, loc='lower right', ncol=2)


    fig.subplots_adjust(hspace=0)
    return fig

# def plot_mid_flux_vs_height_caseB( 
#         snaps_mr,  
#         xx_mr, 
#         mass_labels,  
#         ff_arr = [5,9],
#         ymin = None, ymax = None, 
#         xscale='log', ax0_text = None, ax1_text=None, xx_text = 0.02, yy_text = 0.02, 
#         leg_anchor = (1, 0),
#         colors=colors, legax=[1,1], all_fmt = ['-', '-'],
# ):

#     all_snaps = np.array([snaps_mr, ])
#     all_xx = np.array([ xx_mr, ])

#     fig, axs = plot.figax_single(sharex=True,
#                         xscale=xscale, nrows=2, height=6)
        
#     axs[1].set_xlabel('$z$ (au)')

#     # second row: ratio
#     for ii, ax in enumerate(axs[:]):
#         ff = ff_arr[ii]

#         for jj, snaps in enumerate(all_snaps):
#             xx = all_xx[jj]
#             fmt = all_fmt[1]
#             labels = mass_labels

#             yy = np.zeros((len(snaps), len(xx)))
#             for ss, snap in enumerate(snaps):
#                 temp = snap.flux_prof_ratio
#                 yy[ss] = temp[ff,:-1]
#             draw_flux_vs_height(ax, xx, yy, labels=labels, fmt=fmt, colors=colors, alpha=0.9)

#         ylabel = r'%s GHz $S_\nu$[mJy]' % str(freqs[ff]/10**9)
#         ax.set_ylabel(ylabel)
#         if ax1_text is not None:
#             ax.text(xx_text, yy_text, ax1_text, transform=ax.transAxes, 
#                     weight='bold', horizontalalignment='right')

#         if ymin is not None and ymax is not None:
#             ax.set_ylim(ymin, ymax)
#     # axs[1,1].sharey(axs[1,0])

#     axs[1].legend(bbox_to_anchor = leg_anchor, bbox_transform=ax.transAxes, loc='lower right', ncol=2)

#     # axs[legax[0], legax[1]].legend(bbox_to_anchor = leg_anchor, bbox_transform=ax.transAxes, loc='lower right')

#     fig.subplots_adjust(hspace=0)
#     return fig
    

In [ ]:
fig = plot_mid_flux_vs_height_case( case='A',
        snaps_mr=snaps[1:], xx_mr=zmid_mr[:-1], 
        mass_labels=masss[1:],
        xscale='linear', ax0_text=None, ax1_text=None,
        ymin = 10**-7, ymax = 10**2.5, 
        xx_text=0.99, yy_text=0.025, 
        leg_anchor=(1, 0.0), colors=colors)
fig.savefig('/Users/emigardiner/VICO/pjams-ionization/figures/flux_profiles/midres_fluxprofile_CaseA.png', dpi=200,
            bbox_inches='tight')

In [ ]:
fig = plot_mid_flux_vs_height_case( case='B',
        snaps_mr=snaps[1:], xx_mr=zmid_mr[:-1], 
        mass_labels=masss[1:],
        xscale='linear', ax0_text=None, ax1_text=None,
        ymin = 10**-7, ymax = 10**-1, 
        xx_text=0.99, yy_text=0.025, 
        leg_anchor=(1, 0.0), colors=colors)
fig.savefig('/Users/emigardiner/VICO/pjams-ionization/figures/flux_profiles/midres_fluxprofile_CaseB.png', dpi=200,
            bbox_inches='tight')

### include 1.4 Msun, 4000 yr

In [ ]:
fig = plot_mid_flux_vs_height_case( case='A',
        snaps_mr=snaps[:], xx_mr=zmid_mr[:-1], 
        mass_labels=masss[:],
        xscale='linear', ax0_text=None, ax1_text=None,
        ymin = 10**-7, ymax = 10**2.5, 
        xx_text=0.99, yy_text=0.025, 
        leg_anchor=(1, 0.0), colors=plot.COLORS_INC4)
fig.savefig('/Users/emigardiner/VICO/pjams-ionization/figures/flux_profiles/midres_fluxprofile2_CaseA.png', dpi=200,
            bbox_inches='tight')

In [ ]:
fig = plot_mid_flux_vs_height_case( case='B',
        snaps_mr=snaps[:], xx_mr=zmid_mr[:-1], 
        mass_labels=masss[:],
        xscale='linear', ax0_text=None, ax1_text=None,
        ymin = 10**-7, ymax = 10**-1, 
        xx_text=0.99, yy_text=0.025, 
        leg_anchor=(1, 0.0), colors=plot.COLORS_INC4)
fig.savefig('/Users/emigardiner/VICO/pjams-ionization/figures/flux_profiles/midres_fluxprofile2_CaseB.png', dpi=200,
            bbox_inches='tight')

## Plot mid res flux vs height all

In [ ]:
def plot_mid_flux_vs_height( 
        snaps_mr,  
        xx_mr, 
        mass_labels,  
        ff_arr = [5,9],
        ymin0 = None, ymax0 = None, ymin1 = None, ymax1 = None, 
        xscale='log', ax0_text = None, ax1_text=None, xx_text = 0.02, yy_text = 0.02, 
        leg_anchor = (1, 0),
        colors=colors, legax=[1,1], all_fmt = ['-o', '-o'],
):
    fig, axs = plot.figax(sharex=True,
                         xscale=xscale, nrows=2, ncols=2, figsize=(11,6))

    all_snaps = np.array([snaps_mr, ])
    all_xx = np.array([ xx_mr, ])
    # all_fmt = np.array(['-o',])
    # all_labels = np.array([labels_mr,])
    none_labels = np.repeat('', len(snaps_mr))


    # first row: no ratio
    for ii, ax in enumerate(axs[0,:]):
        ff = ff_arr[ii]

        for jj, snaps in enumerate(all_snaps):
            xx = all_xx[jj]
            fmt = all_fmt[0]
            labels = none_labels

            yy = np.zeros((len(snaps), len(xx)))
            for ss, snap in enumerate(snaps):
                temp = snap.flux_prof
                yy[ss] = temp[ff,:-1]
            draw_flux_vs_height(ax, xx, yy, labels=labels, fmt=fmt, colors=colors,)

        ylabel = r'$S_\nu$ (%s GHz) [mJy]' % str(freqs[ff]/10**9)
        ax.set_ylabel(ylabel)
        if ax0_text is not None:
            ax.text(xx_text, yy_text, ax0_text, transform=ax.transAxes, 
                    weight='bold', horizontalalignment='right')
    if ymin0 is not None and ymax0 is not None:
        axs[0,0].set_ylim(ymin0, ymax0)
    axs[0,1].sharey(axs[0,0])

    # second row: ratio
    for ii, ax in enumerate(axs[1,:]):
        ff = ff_arr[ii]

        for jj, snaps in enumerate(all_snaps):
            xx = all_xx[jj]
            fmt = all_fmt[1]
            if ii==1 and jj==0: 
                labels = mass_labels
            else: labels = none_labels

            yy = np.zeros((len(snaps), len(xx)))
            for ss, snap in enumerate(snaps):
                temp = snap.flux_prof_ratio
                yy[ss] = temp[ff,:-1]
            draw_flux_vs_height(ax, xx, yy, labels=labels, fmt=fmt, colors=colors)

        ylabel = r'$S_\nu$ (%s GHz) [mJy]' % str(freqs[ff]/10**9)
        ax.set_ylabel(ylabel)
        ax.set_xlabel('$z$ (au)')
        if ax1_text is not None:
            ax.text(xx_text, yy_text, ax1_text, transform=ax.transAxes, 
                    weight='bold', horizontalalignment='right')

    if ymin1 is not None and ymax1 is not None:
        axs[1,0].set_ylim(ymin1, ymax1)
    axs[1,1].sharey(axs[1,0])

    axs[1,1].legend(bbox_to_anchor = leg_anchor, bbox_transform=ax.transAxes, loc='lower right')

    # axs[legax[0], legax[1]].legend(bbox_to_anchor = leg_anchor, bbox_transform=ax.transAxes, loc='lower right')

    fig.tight_layout()
    return fig
    

In [ ]:
fig = plot_mid_flux_vs_height( 
        snaps_mr=snaps[1:], xx_mr=zmid_mr[:-1], 
        mass_labels=masss[1:],
        xscale='linear', ax0_text='no cooling', ax1_text='with cooling',
        ymin0 = 10**-7, ymax0 = 10**2.5, ymin1 = 10**-7, ymax1 = 10**-1, 
        xx_text=0.99, yy_text=0.025, 
        leg_anchor=(1, 0.06), colors=colors)
# fig.axes[4].legend(bbox_to_anchor = leg_anchor, bbox_transform=ax.transAxes, loc='lower right')
# fig.legend(bbox_to_anchor=(0.975,0.41))
fig.tight_layout()
fig.savefig('/Users/emigardiner/VICO/pjams-ionization/figures/flux_profiles/midres_fluxprofile_lin.png', dpi=200,
            bbox_inches='tight')
# fig.text(0,0, norm_label, fontsize=14)
# fig.tight_layout()
# fig.savefig('/Users/emigardiner/VICO/pjams-ionization/figures/flux_profiles/allres_fluxprofile_norm_lin.png', dpi=200)

In [ ]:
fig = plot_mid_flux_vs_height( 
        snaps_mr=snaps[1:], xx_mr=zmid_mr[:-1], 
        mass_labels=masss[1:],
        xscale='log', ax0_text='no cooling', ax1_text='with cooling',
        ymin0 = 10**-7, ymax0 = 10**2.5, ymin1 = 10**-7, ymax1 = 10**-1, 
        xx_text=0.99, yy_text=0.025, legax = [1,0],
        leg_anchor=(1, 0.06), colors=colors)

# fig.legend(bbox_to_anchor=(0.075,0.11), loc='lower left')
fig.savefig('/Users/emigardiner/VICO/pjams-ionization/figures/flux_profiles/midres_fluxprofile_log.png', dpi=200)

# fig.text(0,0, norm_label, fontsize=14)
# fig.tight_layout()
